In [ ]:
import pandas as pd
import numpy as np
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup as bs

# Chinese

In [46]:
def scrape_zongheng(url):
    response = requests.get(url)
    soup = bs(response.content, "html.parser")
    title = soup.find("div", attrs={"class": "book-info--title"}).get_text(strip=True)
    div_tags = soup.find("div", attrs={"class": "book-info--tags"})
    tags = []
    serial_status = None
    category = None
    for span in div_tags.find_all("span"):
        try:
            if span["class"][0] == "serialStatus":
                # serial_status = "ongoing" if span.get_text(strip=True) == "连载中" else "completed"
                serial_status = span.get_text(strip=True)
                tags.append(serial_status)
            elif span["class"][0] == "cateFineId":
                category = span.get_text(strip=True)
                tags.append(category)
        except KeyError:
            tags.append(span.get_text(strip=True))
    recent_chapter_info = soup.find(
        "div", attrs={"class": "book-info--chapter-name"}
    ).find(
        "a", attrs={"class": "global-hover"}
    )
    word_count = recent_chapter_info["title"].split("字数：")[1].split()[0]
    author = soup.find("a", attrs={"class": "author-info--name"}).get_text(strip=True)
    return {
        "title": title,
        "author": author,
        "tags": tags,
        "category": category,
        "serial_status": serial_status,
        "word_count": word_count,
        "publication_date": None,
        "synopsis": None,
    }

scrape_zongheng("https://www.zongheng.com/detail/325639")

{'title': '星辰之主',
 'author': '减肥专家',
 'tags': ['连载中', '科幻', '星空', '未来高武', '伪科幻'],
 'category': '科幻',
 'serial_status': '连载中',
 'word_count': '7520169',
 'publication_date': None,
 'synopsis': None}

In [54]:
def scrape_zongheng_with_intro(url):
    options = Options()
    options.add_argument("--window-size=1920,1200")
    options.add_argument('--headless')
    options.add_argument("--silent")

    try:
        with webdriver.Chrome(options=options) as driver:
            driver.get(url)
            page_source = driver.page_source
    except Exception as e:
        print(f"Error while getting page source from {url}. Error: {e}")
        raise e
    soup = bs(page_source, "html.parser")
    title = soup.find("div", attrs={"class": "book-info--title"}).get_text(strip=True)
    div_tags = soup.find("div", attrs={"class": "book-info--tags"})
    tags = []
    serial_status = None
    category = None
    for span in div_tags.find_all("span"):
        try:
            if span["class"][0] == "serialStatus":
                # serial_status = "ongoing" if span.get_text(strip=True) == "连载中" else "completed"
                serial_status = span.get_text(strip=True)
                tags.append(serial_status)
            elif span["class"][0] == "cateFineId":
                category = span.get_text(strip=True)
                tags.append(category)
        except KeyError:
            tags.append(span.get_text(strip=True))
    recent_chapter_info = soup.find(
        "div", attrs={"class": "book-info--chapter-name"}
    ).find(
        "a", attrs={"class": "global-hover"}
    )
    word_count = recent_chapter_info["title"].split("字数：")[1].split()[0]
    author = soup.find("a", attrs={"class": "author-info--name"}).get_text(strip=True)
    synopsis = soup.find("section", attrs={"class": "detail-work-info--introduction"})
    for br_tag in synopsis.find_all("br"):
        br_tag.replace_with("\n")
    synopsis = synopsis.get_text()
    return {
        "title": title,
        "author": author,
        "tags": tags,
        "category": category,
        "serial_status": serial_status,
        "word_count": word_count,
        "publication_date": None,
        "synopsis": synopsis,
    }

scrape_zongheng_with_intro("https://www.zongheng.com/detail/325639")

{'title': '星辰之主',
 'author': '减肥专家',
 'tags': ['连载中', '科幻', '星空', '未来高武', '伪科幻'],
 'category': '科幻',
 'serial_status': '连载中',
 'word_count': '7520169',
 'publication_date': None,
 'synopsis': '世纪之交，人类懵懂着踏入星空，就此暴露在诸神的视线之下。少年罗南背负着祖父的罪孽，走出实验室，且看他：\n高举燃烧的笔记，脚踏诸神的尸骨；\n书写万物的格式，增删宇宙的星图。\n当知：万物皆备于我；必信：吾心即是宇宙。'}

In [55]:
def scrape_jjwxc(url):
    response = requests.get(url)
    soup = bs(response.content, "html.parser")
    title = soup.find("span", attrs={"itemprop": "articleSection"}).get_text(strip=True)
    tags = soup.find("span", attrs={"itemprop": "genre"}).get_text(strip=True).split("-")
    serial_status = soup.find("span", attrs={"itemprop": "updataStatus"}).get_text(strip=True)
    word_count = int(soup.find("span", attrs={"itemprop": "wordCount"}).get_text(strip=True)[:-1])
    tags.append(serial_status)
    extra_tags = soup.find_all("div", attrs={"class": "smallreadbody"})
    for div in extra_tags:
        if div.find("span") is not None:
            extra = div.find_all("a", attrs={"style": "text-decoration:none;color: red;"})
            for a in extra:
                tags.append(a.get_text(strip=True))
    author = soup.find("span", attrs={"itemprop": "author"}).get_text(strip=True)
    first_chapter_row = soup.find("tr", attrs={"itemprop": "chapter"})
    published_at = first_chapter_row.find_all("td")[-1]["title"].split("章节首发时间：")[1]
    synopsis = soup.find("div", attrs={"id": "novelintro"})
    for br_tag in synopsis.find_all("br"):
        br_tag.replace_with("\n")
    synopsis = synopsis.get_text()
    return {
        "title": title,
        "author": author,
        "tags": tags,
        "serial_status": serial_status,
        "category": None,
        "word_count": word_count,
        "published_at": published_at,
        "synopsis": synopsis,
    }

scrape_jjwxc("https://www.jjwxc.net/onebook.php?novelid=6079968")

{'title': '见春天',
 'author': '纵虎嗅花',
 'tags': ['原创', '言情', '近代现代', '爱情', '完结', '花季雨季', '情有独钟', '阴差阳错', '校园', '正剧'],
 'serial_status': '完结',
 'category': None,
 'word_count': 191449,
 'published_at': '2021-09-07 12:00:00',
 'synopsis': '【实体书已出版，新增3000字番外】\n\n--\n\u3000\u3000十五岁的那个夏天，江渡第一次遇见魏清越时，他被一群小混混堵在巷子里暴打，最后，两人一起进了局子。\n\n\u3000\u3000后来，江渡才知道，打魏清越最凶的那个，是他的亲爸。\n\n\u3000\u3000孤独的少女，爱上孤独的少年，自然而然，成为那个夏天最隐蔽的秘密。\n\n\u3000\u3000“而今，在梅中，我遇到的那个人，终会长大，我跟他的所有，皆成文字，北国正芳春。”\n\n\n--\n文中分享的几首歌，希望大家喜欢，也希望大家能够安静欣赏。'}

In [10]:
def scrape_17k(url):
    response = requests.get(url)
    soup = bs(response.content, "html.parser")

In [11]:
def scrape_qidian(url):
    response = requests.get(url)
    soup = bs(response.content, "html.parser")

# Japanese

In [57]:
syosetu_genre_map = {
    0: "未選択〔未選択〕",
    101: "異世界〔恋愛〕",
    102: "現実世界〔恋愛〕",
    201: "ハイファンタジー〔ファンタジー〕",
    202: "ローファンタジー〔ファンタジー〕",
    301: "純文学〔文芸〕",
    302: "ヒューマンドラマ〔文芸〕",
    303: "歴史〔文芸〕",
    304: "推理〔文芸〕",
    305: "ホラー〔文芸〕",
    306: "アクション〔文芸〕",
    307: "コメディー〔文芸〕",
    401: "VRゲーム〔SF〕",
    402: "宇宙〔SF〕",
    403: "空想科学〔SF〕",
    404: "パニック〔SF〕",
    9901: "童話〔その他〕",
    9902: "詩〔その他〕",
    9903: "エッセイ〔その他〕",
    9904: "リプレイ〔その他〕",
    9999: "その他〔その他〕",
    9801: "ノンジャンル〔ノンジャンル〕",
}

syosetsu_nocgenre_map = {
    1: "ノクターンノベルズ(男性向け)",
    2: "ムーンライトノベルズ(女性向け)",
    3: "ムーンライトノベルズ(BL)",
    4: "ミッドナイトノベルズ(大人向け)",
}

def syosetsu(ncode):
    response = requests.get(f"https://api.syosetu.com/novelapi/api/?out=json&ncode={ncode}").json()[1]
    return_me = {
        "title": response["title"],
        "author": response["writer"],
        "tags": response["keyword"].split(),
        "category": syosetu_genre_map[response["genre"]],
        "serial_status": "連載中" if response["novel_type"] == 1 and response["end"] == 1 else "完結済",
        "word_count": response["length"],
        "published_at": response["general_firstup"],
        "chapters": response["general_all_no"],
        "synopsis": response["story"],
    }
    return_me["tags"].append(return_me["category"])
    return return_me

def syosetsu_r18(ncode):
    response = requests.get(f"https://api.syosetu.com/novel18api/api/?out=json&ncode={ncode}").json()[1]
    return_me = {
        "title": response["title"],
        "author": response["writer"],
        "tags": response["keyword"].split(),
        "category": syosetsu_nocgenre_map[response["nocgenre"]],
        "serial_status": "連載中" if response["novel_type"] == 1 and response["end"] == 1 else "完結済",
        "word_count": response["length"],
        "published_at": response["general_firstup"],
        "chapters": response["general_all_no"],
        "synopsis": response["story"],
    }
    return_me["tags"].append(return_me["category"])
    return return_me

In [58]:
syosetsu("n7466jx")

{'title': '異界冒険譚',
 'author': 'とーふ',
 'tags': ['異世界転移',
  'ギャグ',
  'シリアス',
  'ほのぼの',
  '男主人公',
  '冒険',
  '日常',
  'ハッピーエンド',
  'ラブコメ',
  'バトル',
  '聖女',
  'ハイファンタジー〔ファンタジー〕'],
 'category': 'ハイファンタジー〔ファンタジー〕',
 'serial_status': '連載中',
 'word_count': 849421,
 'published_at': '2025-01-01 18:10:00',
 'chapters': 273,
 'synopsis': '少年の願いはただ一つ。\n妹の幸福であった。\n\n病弱な妹が、\n自分の足で歩いて、笑顔で日々を送る。\n\nただ、それだけが少年の願いであった。\n\n故に……挑む。\n自らが住む世界とは異なる世界。\n\n異世界へと――\n'}

In [59]:
# Short stories only have 1 chapter and their serial status will just be marked as completed
syosetsu("n1145lc")

{'title': 'とんずらした兄のことは許さないけれど、兄の元婚約者と幸せな家庭を築いた僕の話。',
 'author': '葵ふたば',
 'tags': ['アイリスIF7大賞',
  '貴族',
  '鉄道',
  '恋愛',
  '結婚',
  '婚約破棄',
  '駆け落ち',
  'プロポーズ',
  '円満',
  '異世界〔恋愛〕'],
 'category': '異世界〔恋愛〕',
 'serial_status': '完結済',
 'word_count': 18632,
 'published_at': '2025-09-19 09:53:42',
 'chapters': 1,
 'synopsis': '\n侯爵家の三男として生まれた僕は、国営の鉄道会社に就職して機関士としての人生を歩んでいた。\nけれどもある日突然、実家へと連れ戻されてしまう。\nどうやら次男である僕の兄が、婚約者の居る身でありながら平民の女と駆け落ちしたのだという。\n僕は兄に代わって父の持つ爵位のひとつを継承するどころか、兄の元婚約者のお嬢さんとも仲を深めなくてはならなくなった。\n\nお嬢さんの家族は気さくで優しいし、なんと言っても、お嬢さんの実家周りには私設のトロッコ列車が走っている。鉄道好きとしてこれを堪能しない手はない！\n\nでもお嬢さんは、\n何もかもが付き焼き刃でハリボテ貴族な僕なんかと結婚するには、申し訳ないくらいに素敵な女性だ。\n身勝手な兄につけられた心の傷は深いだろう。\nだからこそ、彼女には幸せな未来を選び取って欲しい…。\n\nこれは煤ぶった僕が色んな人から尻を叩かれながらも、素敵なお嬢さんと幸福に至るひと欠片のお話。\n\n'}

In [60]:
syosetsu_r18("n6752ch")

{'title': '王太子妃になんてなりたくない！！',
 'author': '月神サキ',
 'tags': ['残酷な描写あり',
  '異世界転生',
  '異世界転移',
  'ハッピーエンド',
  'R15',
  'ファンタジー',
  '恋愛',
  '美形',
  '女性視点',
  '処女',
  '王子',
  '執着',
  'らぶえっち',
  '溺愛',
  '男性視点',
  '腹黒',
  'ときどきコメディ',
  '絶倫',
  'ムーンライトノベルズ(女性向け)'],
 'category': 'ムーンライトノベルズ(女性向け)',
 'serial_status': '連載中',
 'word_count': 2883021,
 'published_at': '2014-09-24 16:58:30',
 'chapters': 814,
 'synopsis': '※コミカライズ連載中！\u3000\n※黒木捺先生版の王太子妃コミカライズは、2022/10/28連載終了。\n※続編『婚約者編』が、鴨野れな先生作画で2022/11/25より始まりました。ゼロサムオンラインにて毎月第四金曜日更新です。\n※黒木捺先生の無印版1～3巻（完結）がゼロサムコミックスにて発売中です。\n※鴨野れな先生作画の『婚約者編』1～3巻も発売中です。\n※小説版は、一迅社様メリッサレーベルより四六版、文庫版、各全十巻が発売中です。\n※続編『王太子妃編』全十巻が四六版、文庫版にて発売中です。\n※更に続編『王妃編』1巻が発売中。2巻は2025／08／01発売予定です。\n\nかつては日本人。転生したと思ったら、公爵令嬢。\nちょっと待って婚約者は王太子？\nいくら美形だろうと、一夫多妻の王族のもとになんて絶対に嫁ぎたくない。\n何とか婚約破棄を考えてリディが思いついたのは、あるとんでもないこと。\n彼女の計画通りことはうまく運ぶのか。\n\n※なんちゃって設定で軽く書いていますので、設定の甘さ等、つっこみは入れない方向でお願いします。\n※『婚約者編』『王太子妃編』完結しました。『王妃編』連載中です。\n\n\n'}